In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from iexfinance.refdata import get_symbols, get_iex_symbols

import os
import sys
import pandas as pd
import altair as alt
import datetime as dt

from pandas import to_datetime
from datetime import datetime, date
from iexfinance.stocks import get_historical_data
from dateutil.relativedelta import relativedelta

In [3]:
# Used to log the `MESSAGES USED` in `_IEXBAse`.
# https://addisonlynch.github.io/iexfinance/stable/logging.html
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
os.environ["IEX_LOG_LEVEL"] = "DEBUG"

DEBUG:root:test


In [4]:
IEX_TOKEN = "Tpk_57fa15c2c86b4dadbb31e0c1ad1db895"
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"
store = pd.HDFStore('dev_iex_store.h5')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1567, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 959, in H5FD_sec2_lock
    unable to lock file, errno = 35, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file 'dev_iex_store.h5'

In [ ]:
symbols = get_symbols(token=IEX_TOKEN)[:10]

In [ ]:
df = get_historical_data('AAPL', datetime.now() - relativedelta(days=60), to_datetime(dt.date.today()), close_only=True, output_format='pandas', token=IEX_TOKEN)

In [ ]:
start = datetime.now() - relativedelta(days=60)
end = datetime.now() - relativedelta(days=1)
symbols = ['AAPL', 'GOOG']
json = get_historical_data(symbols, start, end, close_only=False, output_format='json', token=IEX_TOKEN)
df = get_historical_data(symbols, start, end, close_only=False, output_format='pandas', token=IEX_TOKEN)

In [ ]:
pd.DataFrame.from_dict(json['AAPL']['chart']).columns

In [ ]:
def json_to_pandas(out):
    if len(symbols) > 1:
        out = {
            (symbol, day["date"]): day
            for symbol in out
            for day in out[symbol]["chart"]
        }
#         result = pd.DataFrame.from_dict(out, orient='index')
        result = pd.DataFrame.from_dict(out, orient="columns").drop("date").T
        result.index = result.index.set_levels(
            [result.index.levels[0], pd.to_datetime(result.index.levels[1])]
        )
        idx = pd.IndexSlice
        result = result.loc[idx[:, start : end], :]
    else:
        out = {entr["date"]: entr for entr in out[symbols[0]]["chart"]}
        result = pd.DataFrame.from_dict(out, orient="columns").drop("date").T
        result.index = pd.to_datetime(result.index)
        result = result.loc[start : end, :]
    return result

def json_to_pandas1(out):
    if len(symbols) > 1:
        out = {
            (symbol, pd.to_datetime(day["date"])): day
            for symbol in out
            for day in out[symbol]["chart"]
        }
        result = pd.DataFrame.from_dict(out, orient='index')
        idx = pd.IndexSlice
        result = result.loc[idx[:, start : end], :]
    else:
        out = {entr["date"]: entr for entr in out[symbols[0]]["chart"]}
        result = pd.DataFrame.from_dict(out, orient="columns").drop("date").T
        result.index = pd.to_datetime(result.index)
        result = result.loc[start : end, :]
    return result

In [ ]:
import time

start_time = time.time()
json_to_pandas(json)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
json_to_pandas1(json)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
all_symbols = get_symbols(token=IEX_TOKEN)
symbols_df = all_symbols[:10]
symbols = [symbol_metadata['symbol'] for _, symbol_metadata in symbols_df.iterrows()]
symbols